In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import os
from processingmm import batch_processing

 Could not import win32api and/or win32con


In [8]:
from histopolalign.AlignImages import combine_images, prepare_images, semi_automatic_tool, align_folders, match_skeletons, align_imgs, finalize_mask, get_numerical_results

# 1. Process the measurements that will be aligned

In [9]:
# get the folder in which the polarimetric measurements are stored
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
neoplastic_folder = '38'
polarimetry_path = os.path.join(neoplastic_polarimetry_path, neoplastic_folder)
directories = [os.path.join(neoplastic_polarimetry_path, neoplastic_folder)]

In [10]:
# process the measurements, if needed
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')
batch_processing.batch_process(directories, calib_directory, run_all = False)

0it [00:00, ?it/s]


[]

# 2. Get the combined masks for the histology folders

In [11]:
# create the folde that will be used to align pathology and polarimetry
combine_images.create_to_align_folder()

# load the pathology images...
histology_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HistologyResults')
imgs_all = combine_images.load_the_images(histology_path, Verbose = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 15.20it/s]

Load the images: 0.270 seconds.


In [12]:
# ...and process them
combine_images.get_combined_img(imgs_all, force_recompute = True, Verbose = True)

Get combined images: 0.819 seconds.


# 3. Obtain the parameters (manually using the GUI) to first align the histology and the polarimetry

### 3.1. Prepare the images

In [13]:
alignment_measurements = prepare_images.create_the_alignments(histology_path, polarimetry_path, Verbose = True)
alignment_measurements

Get polarimetry filenames and rename images: 0.016 seconds.
Create the alignment objects: 0.016 seconds.


[Folder path: C:\Users\romai\Desktop\TestPathologyPaper\PathologyPaper\data\TumorMeasurements\38\2022-09-01_T_HORAO-38-BF_FR_S1_1, histology path : C:\Users\romai\Desktop\TestPathologyPaper\PathologyPaper\data\HistologyResults\38\HORAO-38-S1-1,  status: initialized]

### 3.2. Load the polarimetry, labels and histology (H&E) images for each polarimetry folder

In [14]:
alignment_measurements = prepare_images.load_and_preprocess_imgs(alignment_measurements, force_recompute = False, Verbose = True)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Get the images and center of mass: 0.020 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.05it/s]

Load the images: 0.226 seconds.


### 3.3. Semi-automatic rotation/alignement tool

In [16]:
for measurement in alignment_measurements:
    selector = semi_automatic_tool.ask_for_parameters(measurement, force_recompute = False)

# 4. Actually perform the alignment

In [17]:
align_folders.create_align_folders(alignment_measurements, Verbose = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.33it/s]

Process the images to apply the selected changes: 0.314 seconds.
Save the processed images: 0.033 seconds.


### 4.1. Automatic part

In [18]:
border_parameter = 5
nsamples = 400
max_distance = 150

for measurement in tqdm(alignment_measurements):
    match_skeletons.match_skeletons(measurement, border_parameter, nsamples = nsamples, max_distance = max_distance,
                                                                   Verbose = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.14s/it]

Match the two skeletons: 16.144 seconds.


### 4.2. Semi-automatic part

In [19]:
align_folders.semi_automatic_processing(alignment_measurements)

In [44]:
alignment_measurements = align_imgs.align_img_master(alignment_measurements)

### 4.3. Get the final masks and overlay them on the polarimetric parameter maps

In [24]:
finalize_mask.generate_final_masks(alignment_measurements, Verbose = True)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

create_the_image_labels_propagated: 7.111 seconds.
get_final_mask: 3.395 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.51s/it]

save_all_combined_maps: 1.390 seconds.


### 4.4. Finalize and save the numerical results

In [25]:
values = get_numerical_results.create_numerical_values(alignment_measurements)